In [1]:
import numpy as np
import libpysal
import spreg
import geopandas as gpd
import polars as pl
from local_functions import *

In [2]:
territorial_information = gpd.read_file("data/transformed/territorial_information_summary.geojson")

In [3]:
for c in territorial_information.columns:
    print(c)

Teritoriālā vienība
Platība (km^2)
Gads
Iedzīvotāju skaits gada sākumā
Gada vidējais iedzīvotāju skaits
Iedzīvotāju dabiskais pieaugums
Migrācijas saldo
Iedzīvotāju skaita izmaiņas - pavisam
Mēneša darba samaksas mediāna
Iedzīvotāji darbspējas vecumā
Pašvaldības izdevumi: Vispārējie valdības dienesti (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Vispārējie valdības dienesti (Kapitālie izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Kapitālie izdevumi)
Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Teritoriju un mājokļu apsaimniekošana (Kapitālie izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Atpūta, kultūra un reliģija (Kapitālie izdevumi)
Pašvaldības izdevumi: Izglītība (Uzturēšanas izdevumi)
Pašvaldības izdevumi: Izgl

In [24]:
y_col = ['Bezdarba līmenis']
x_col = [
    'Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Ekonomiskā darbība (Kapitālie izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Izglītība (Kapitālie izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Pašvaldības izdevumi: Sociālā aizsardzība (Uzturēšanas izdevumi) (uz 1 iedz.)',
    'Autoceļu un pašvaldību ielu garums (asfaltbetons) (uz 1 iedz.)',
    'Ekonomiski aktīvi uzņēmumi (uz 1000 iedz.)',
    'Iedzīvotāju īpatsvars ar augstāko izglītību',
]

columns_to_use = [
    'Teritoriālā vienība',
    'Gads',
    'geometry'
] + x_col + y_col

In [25]:
(
    pl.from_dataframe(territorial_information.drop("geometry", axis=1))
    .unpivot(
        on=y_col + x_col,
        index=['Teritoriālā vienība', 'Gads']
    )
    .filter(~pl.col("value").is_null())
    .group_by("variable")
    .agg(
        pl.col("Gads").min().alias("Min year"),
        pl.col("Gads").max().alias("Max year")
    )
    .group_by(1)
    .agg(
        pl.col("Min year").max(),
        pl.col("Max year").min()
    )
)

literal,Min year,Max year
i32,i32,i32
1,2021,2023


In [26]:
df = territorial_information[columns_to_use]
df = df[df.Gads.isin([2021, 2022, 2023])]
df = df.fillna(0)
df = df.sort_values(by=['Teritoriālā vienība', 'Gads'])

In [27]:
t = df[df.Gads==2023]
w = libpysal.weights.Queen.from_dataframe(t, ids="Teritoriālā vienība")
w.transform = 'r'

In [28]:
df[x_col]

,Pašvaldības izdevumi: Ekonomiskā darbība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Ekonomiskā darbība (Kapitālie izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Izglītība (Kapitālie izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Sabiedriskā kārtība un drošība (Uzturēšanas izdevumi) (uz 1 iedz.),Pašvaldības izdevumi: Sociālā aizsardzība (Uzturēšanas izdevumi) (uz 1 iedz.),Autoceļu un pašvaldību ielu garums (asfaltbetons) (uz 1 iedz.),Ekonomiski aktīvi uzņēmumi (uz 1000 iedz.),Iedzīvotāju īpatsvars ar augstāko izglītību
323,46.911465,185.103347,79.583069,8.454388,179.703851,0.014949,67.0,21.3
366,81.202168,80.784374,48.336396,9.327414,239.781552,0.015109,69.0,21.9
409,108.193748,95.787599,17.310674,11.978169,276.433264,0.016269,68.0,22.1
301,76.760046,49.767838,261.387490,6.369598,165.659621,0.017098,77.0,21.5
344,99.745539,236.518729,702.290296,7.039522,263.975962,0.018803,82.0,22.2
...,...,...,...,...,...,...,...,...
375,22.377051,0.465230,353.122676,34.858093,115.138489,0.007656,88.0,46.2
418,22.238807,2.347291,54.610572,37.970078,141.943583,0.007250,85.0,46.8
338,86.447817,110.941450,310.798617,54.775443,166.908335,0.006184,81.0,39.8
381,88.546219,93.966395,100.792055,44.668794,215.914204,0.006053,86.0,40.8


In [29]:
Y_df = df[y_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[y_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [30]:
X_df = df[x_col+["Teritoriālā vienība"]].groupby("Teritoriālā vienība")[x_col].apply(lambda df: df.reset_index(drop=True)).unstack()

In [31]:
Y = Y_df.to_numpy()
X = X_df.to_numpy()

fe_lag = spreg.Panel_FE_Lag(Y, X, w, name_y = y_col, name_x=x_col, name_ds="df")

In [32]:
print(fe_lag.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: MAXIMUM LIKELIHOOD SPATIAL LAG PANEL - FIXED EFFECTS
-----------------------------------------------------------------------
Data set            :          df
Weights matrix      :     unknown
Dependent Variable  :Bezdarba līmenis                Number of Observations:         129
Mean dependent var  :      0.0000                Number of Variables   :           9
S.D. dependent var  :      0.5510                Degrees of Freedom    :         120
Pseudo R-squared    :      0.6632
Spatial Pseudo R-squared:  0.6285
Sigma-square ML     :       0.102                Log likelihood        :    -352.009
S.E of regression   :       0.319                Akaike info criterion :     722.018
                                                 Schwarz criterion     :     747.756

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
-------------

In [63]:
name_y = ["HR70", "HR80", "HR90"]
y = np.array([db.by_col(name) for name in name_y]).T

In [66]:
y

array([[ 0.        ,  8.85582713,  0.        ],
       [ 0.        , 17.20874204, 15.88562351],
       [ 1.91515848,  3.4507747 ,  6.46245315],
       ...,
       [ 6.02355209,  5.2803488 ,  4.36732988],
       [ 1.80014761,  3.00003   ,  3.72771194],
       [ 2.28487867,  1.19474313,  2.04885495]])